# Histogram Plot over time qualitative analysis

In [3]:
# Install Dependencies
#!pip install scikit-image
#!pip install scenedetect[opencv,progress_bar]

In [4]:
import pandas as pd
import numpy as np
import io
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skimage import color

import cv2

yt_url_prefix = 'https://www.youtube.com/watch?v='
dataset_folder = "../../data"

In [5]:
downloaded = os.listdir(dataset_folder)

In [6]:
def plotColor3D(im, rgb_to_space, space_to_rgb, axes="XYZ"):
    """Expects an input image from openCV of channel order BGR.
    Expects two skimage functions to map to and from RGB and desired color space."""
    im = np.flip(im, axis=2) # Switch BGR (openCV standard) to RGB
    im = rgb_to_space(im) # Convert RGB to space for different viewing
    
    H, _ = np.histogramdd(
        im.reshape(-1, im.shape[-1]),  #flattened image
        bins=(255, 255, 255)
    )

    # Determine x, y, z pairs of points to plot
    nz = H.nonzero()
    xs, ys, zs = nz
    
    # Determing colors of those points
    colors = np.array(nz).T
    colors_rgb = np.array([colors], dtype='uint8')
    colors_rgb = space_to_rgb(colors_rgb)[0]
    #colors_rgb *= 255
    #colors_rgb = colors.astype('uint8')
    
    # Determine size of points
    sizes = H[H.nonzero()]
    sizes = np.log(sizes) * 10

    # Plot figure
    fig = plt.figure(figsize=(20, 20))
    # Axis ordering 1
    ax = fig.add_subplot(221, projection='3d')
    _ = ax.scatter(
        xs = xs,
        ys = ys,
        zs = zs,
        s = sizes,
        c = colors_rgb
    )
    ax.set_xlabel(axes[0], fontsize=20)
    ax.set_ylabel(axes[1], fontsize=20)
    ax.set_zlabel(axes[2], fontsize=20)
    ax.set_xlim3d(0, 255)
    ax.set_ylim3d(0, 255)
    ax.set_zlim3d(0, 255)
    # Axis ordering 2
    ax = fig.add_subplot(222, projection='3d')
    _ = ax.scatter(
        xs = ys,
        ys = zs,
        zs = xs,
        s = sizes,
        c = colors_rgb
    )
    ax.set_xlabel(axes[1], fontsize=20)
    ax.set_ylabel(axes[2], fontsize=20)
    ax.set_zlabel(axes[0], fontsize=20)
    ax.set_xlim3d(0, 255)
    ax.set_ylim3d(0, 255)
    ax.set_zlim3d(0, 255)
    # Axis ordering 3
    ax = fig.add_subplot(223, projection='3d')
    _ = ax.scatter(
        xs = zs,
        ys = xs,
        zs = ys,
        s = sizes,
        c = colors_rgb
    )
    ax.set_xlabel(axes[2], fontsize=20)
    ax.set_ylabel(axes[0], fontsize=20)
    ax.set_zlabel(axes[1], fontsize=20)
    ax.set_xlim3d(0, 255)
    ax.set_ylim3d(0, 255)
    ax.set_zlim3d(0, 255)
    # Save Figure
    fig.savefig('hist.png', dpi = 50)
    hist_rgb = cv2.imread('hist.png').astype('uint8')
    plt.close('all')

    return H, hist_rgb


In [7]:
def make_rgbt_hists(video_file_name):
    vid = cv2.VideoCapture(video_file_name)
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    vid_file = os.path.basename(video_file_name)
    yt_id = vid_file.split(".")[0]

    vid_rgb = cv2.VideoWriter(f"{yt_id}_rgb.avi", cv2.VideoWriter_fourcc(*"MJPG"), 23.98, (1000, 1000))
    vid_hsv = cv2.VideoWriter(f"{yt_id}_hsv.avi", cv2.VideoWriter_fourcc(*"MJPG"), 23.98, (1000, 1000))
    vid_lab = cv2.VideoWriter(f"{yt_id}_lab.avi", cv2.VideoWriter_fourcc(*"MJPG"), 23.98, (1000, 1000))  

    i = 0
    
    while True:
        # Read image and make histogram
        success, image = vid.read()
        
        # Break if no more frames left
        if not success:
            break

        # Print status update if there are more frames to process
        print(f"Processing Frame {i}/{frame_count}", end="\r")   
        
        # Process Histogram
        H_rgb, H_rgb_image = plotColor3D(
            image, 
            lambda x: x,
            lambda x: x.astype('float64')/255,
            'RGB'
        )
        vid_rgb.write(H_rgb_image)
        H_hsv, H_hsv_image = plotColor3D(
            image, 
            color.rgb2hsv,
            color.hsv2rgb,
            'HSV'
        )
        vid_hsv.write(H_hsv_image)
        H_lab, H_lab_image = plotColor3D(
            image, 
            color.rgb2lab,
            color.lab2rgb,
            'LAB'
        )
        vid_lab.write(H_lab_image)

        i += 1
    
    # Close video streams
    vid_rgb.release()
    vid_hsv.release()
    vid_lab.release()
    

In [ ]:
for file in downloaded[1:5]:
    file_dir = os.path.join(dataset_folder, file)
    print(f"Processing Video {file_dir}")
    make_rgbt_hists(file_dir)

Processing Video ../../data/TKPmGjVFbrY.mp4


/home/bmild/anaconda2/envs/comp-color/lib/python3.7/site-packages/skimage/color/colorconv.py:258: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v
/home/bmild/anaconda2/envs/comp-color/lib/python3.7/site-packages/skimage/color/colorconv.py:264: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/home/bmild/anaconda2/envs/comp-color/lib/python3.7/site-packages/skimage/color/colorconv.py:268: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]
/home/bmild/anaconda2/envs/comp-color/lib/python3.7/site-packages/skimage/color/colorconv.py:272: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]
